<a href="https://colab.research.google.com/github/ShinAsakawa/2015corona/blob/master/2023notebooks/2023_0608rinna_chatGPT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_format = 'retina'
import torch
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

if isColab:
    !pip install 'transformers[torch]'
    !pip install sentencepiece

[japanese-gpt-neox-3.6b-instruction-sft-v2](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft-v2)

## 概要 <!-- ## Overview-->

このリポジトリは，36 億個のパラメータを持つ日本語 GPT-NeoX モデルを提供する。
このモデルは rinna/japanese-gpt-neox-3.6b をベースに，命令に従う会話エージェントとして機能するように細調整されている。
<!-- This repository provides a Japanese GPT-NeoX model of 3.6 billion parameters. 
The model is based on rinna/japanese-gpt-neox-3.6b and has been finetuned to serve as an instruction-following conversational agent. -->

このモデルは，従来の SFT モデル rinna/japanese-gpt-neox-3.6b-instruction-sft とは若干異なり，学習に使用するデータ分割が異なっている。
<!--This model slightly differs from the previous SFT model rinna/japanese-gpt-neox-3.6b-instruction-sft, where a different data split is used for training.-->
 
* モデルアーキテクチャ <!-- * Model architecture-->

36層，2816 隠れ層サイズのtransformerベースの言語モデル。
<!--A 36-layer, 2816-hidden-size transformer-based language model.  -->

* SFT と従来の SFT の比較評価 <!-- * SFT vs. previous SFT evaluation-->

本 SFT モデルと従来 SFT モデルの性能差を評価するため，100 プロンプトで ChatGPTベースの自動評価を実施した。
<!-- We conducted ChatGPT-based automated evaluation on 100 prompts to assess the performance difference between this SFT model and the previous SFT model.  -->

|this SFT vs. previous SFT| win|tie|loss|
|:---:|:---:|:---:|:---:|
|ChatGPT auto. evaluation|	55%|	0%	|45%|

* 微調整 <!--Finetuning-->

微調整データは，以下のデータセットの下位集合であり，日本語に翻訳されている。 <!-- The finetuning data is the subset of the following datasets and has been translated into Japanese.-->

* [人間工学的 HH RLHF データ](https://huggingface.co/datasets/Anthropic/hh-rlhf)
* [FLAN Instruction Tuning データ](https://github.com/google-research/FLAN)
* [スタンフォード 人間嗜好データセット (Stanford Human Preferences Dataset)](https://huggingface.co/datasets/stanfordnlp/SHP)

データは公開されない

<!--* Anthropic HH RLHF data
* FLAN Instruction Tuning data
* Stanford Human Preferences Dataset

The data will not be released. -->

* Authors

Tianyu Zhao and Kei Sawada

###  I/O Format

入力を構成するために、特別なフォーマットを採用している。 <!-- A special format has been adopted to construct inputs.-->

* 入力プロンプトは，ユーザーとシステムの会話としてフォーマットされている。
* 各入力発話は，(1) 話者 (`ユーザ` または `システム`)，(2) コロン (`:`)，(3) 空白 (` `)，(4) 発話テキスト(例: `世界で一番高い山は？`)
* 入力プロンプトは `システム` で終了する必要があり，モデルが応答を生成することを許可する。
* 入力と出力に含まれるすべての改行記号を `<NL>` に置き換える必要がある。
* 入力プロンプト内の発言は全て `<NL>` で区切る。

以下は，会話から入力を作成する例である。

<!-- * An input prompt is formatted as a conversation between ユーザー and システム.
* Each input utterance consists of (1) its speaker ("ユーザー" or "システム"), (2) a colon (":"), (3) a whitespace (" "), and (4) utterance text (e.g. "世界で一番高い山は？").
* The input prompt should be ended with "システム: " to acknowledge the model to generate a response.
* Since the model's tokenizer does not recognize "\n", a special newline symbol "<NL>" is used instead.
* All the newlines in input and output utterances should be replaced with "<NL>".
* All the utterances in the input prompt should be separated by "<NL>".
Following is an example to construct an input from a conversation. -->


In [ ]:
prompt = [
    {
        "speaker": "ユーザー",
        "text": "コンタクトレンズを慣れるにはどうすればよいですか？"
    },
    {
        "speaker": "システム",
        "text": "これについて具体的に説明していただけますか？何が難しいのでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "目が痛いのです。"
    },
    {
        "speaker": "システム",
        "text": "分かりました、コンタクトレンズをつけると目がかゆくなるということですね。思った以上にレンズを外す必要があるでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "いえ、レンズは外しませんが、目が赤くなるんです。"
    }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)
print(prompt)
# "ユーザー: コンタクトレンズを慣れるにはどうすればよいですか？<NL>システム: これについて具体的に説明していただけますか？何が難しいのでしょうか？<NL>ユーザー: 目が痛いのです。<NL>システム: 分かりました、コンタクトレンズをつけると目がかゆくなるということですね。思った以上にレンズを外す必要があるでしょうか？<NL>ユーザー: いえ、レンズは外しませんが、目が赤くなるんです。<NL>システム: "


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 訓練済ファイルをダウンロード
# トークナイザのダウンロードして tokenizer に格納
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft-v2", use_fast=False)

# モデル本体のダウンロードとし model に格納
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft-v2")

# GPU の使用
if torch.cuda.is_available():
    model = model.to("cuda")

# 対話文をトークン ID に変換して token_ids に格納
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


with torch.no_grad():
    # モデルにトークン ID を与えて output_ids を得る
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.7,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

#output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
#output = output.replace("<NL>", "\n")
#print(output)


In [ ]:
def get_chatGPT_output(prompt:list):
    """chatGPT に prompt を与えて，出力を得る"""

    # プロンプト文を tokenizer に与えて，トークン ID を得る
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    # トークン ID をモデルに与えて output_ids を得る
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            max_new_tokens=128,
            temperature=0.7,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # output_ids を tokenizer に通してトークンを復号化
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])

    # 復号化した文の <NL> を 改行コードに置き換える
    output = output.replace("<NL>", "\n")
    return output

In [ ]:
# Python では [] で囲まれたデータをリスト list と呼びます。
# {} で囲まれたデータを辞書 dict と呼びます。
# したがって，以下のデータは，5 つの辞書からなるリストを prompt という名前で定義しています。
prompt0 = [
    {
        "speaker": "ユーザー",
        "text": "コンタクトレンズを慣れるにはどうすればよいですか？"
    },
    {
        "speaker": "システム",
        "text": "これについて具体的に説明していただけますか？何が難しいのでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "目が痛いのです。"
    },
    {
        "speaker": "システム",
        "text": "分かりました、コンタクトレンズをつけると目がかゆくなるということですね。思った以上にレンズを外す必要があるでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "いえ、レンズは外しませんが、目が赤くなるんです。"
    }
]
print(prompt0)

In [ ]:
# 上記のリストに対して，`speaker` と `text` というタグを使って，リスト prompt を `:` を使った形式に書き換える
prompt1 = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt0
]
print(prompt1)

In [ ]:
# prompt を `<NL>` で繋ぐ
prompt2 = "<NL>".join(prompt1)
print(prompt2)


In [ ]:
# 最後に `システム:` をつけて，システムの出力を促す。
prompt3 = (
    prompt2
    + "<NL>"
    + "システム: "
)
print(prompt3)

In [ ]:
#token_ids = tokenizer.encode(prompt3, add_special_tokens=False, return_tensors="pt")
#print(token_ids.size())
#print(tokenizer.convert_ids_to_tokens(token_ids.detach().numpy()[0]))

In [ ]:
output = get_chatGPT_output(prompt3)


In [ ]:
print(output)